In [1]:
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, LineString
import os
import math
import json
import gzip
import time
import calendar
from datetime import datetime
import numpy as np
import pandas as pd
from datetime import date
from multiprocessing import Pool
import warnings
warnings.filterwarnings('ignore')
SETS = './sets/' # папка с файлами с наборами
LONG = 0.0159 # долгота расстояние 1 км от точки на восток или на запад
LAT = 0.009 # широта расстояние 1 км от точки на север или на юг

In [2]:
def default_serializer(obj): # для правильного преобразования даты в ISO формат
    if isinstance(obj, (date)):
        return obj.isoformat()
    raise TypeError(f'Object of type {obj.__class__.__name__} is not JSON serializable')
    

def load_dataset(filename): 
    if os.path.exists(SETS + filename + '.gz'):
        with gzip.open(SETS + filename + '.gz', 'rb') as gzip_ref:
            return pd.DataFrame(json.load(gzip_ref))
        
    
def save_dataset(filename):
    data = globals().get(filename)
    if data is not None:
        data = data.to_dict(orient='records')
        json_data = json.dumps(data, ensure_ascii=False, default=default_serializer)
        with gzip.open(SETS + filename + '.gz', 'wb') as gzip_file:
            gzip_file.write(json_data.encode('utf-8'))
        print(f'Сохранено {len(data)} записей в {filename}.gz')
        

def distance(lon1, lat1, lon2, lat2):
    """
    Функция для вычисления расстояния между двумя точками на плоскости по их GPS координатам
    """
    R = 6371  # Радиус Земли в километрах
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) * np.sin(dlat / 2) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) * np.sin(dlon / 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c


def km_min(exams, ecology):
    counter = 0
    km_list = []
    for i, row in ecology.iterrows():
        if exams['datesSet'].intersection(row['datesSet']):
            a_list = exams['geoList']
            for a in range(0, len(a_list), 2):
                a_longlat = a_list[a:a+2]
                if len(a_longlat) >= 2: # если в списке больше двух точек
                    a_long = a_longlat[0]
                    a_lat = a_longlat[1]
                b_list = row['geoList']
                for b in range(0, len(b_list), 2):
                    b_longlat = b_list[b:b+2]
                    if len(b_longlat) >= 2:
                        b_long = b_longlat[0]
                        b_lat = b_longlat[1]
                    km = distance(a_long, a_lat, b_long, b_lat)
                    if km < 1:
                        counter += 1
                        km_list.append(km)
                        return 1-math.prod(km_list)
                        #print(f'расстояние {km} между [ {a_long}, {a_lat} ] и [ {b_long}, {b_lat} ]')
    return 0


# def calculate_distances(exams_row, ecology_rows):
#     counter = 0
#     a_list = exams_row['geoList']
    
#     for b_row in ecology_rows.itertuples(index=False):
#         if exams_row['datesSet'].intersection(b_row.datesSet):
#             b_list = b_row.geoList
#             for a in range(0, len(a_list), 2):
#                 if len(a_list) < 2:
#                     continue
#                 a_long = a_list[a]
#                 a_lat = a_list[a+1]
#                 for b in range(0, len(b_list), 2):
#                     if len(b_list) < 2:
#                         continue
#                     b_long = b_list[b]
#                     b_lat = b_list[b + 1]
#                     km = distance(a_long, a_lat, b_long, b_lat)
#                     if km < 1:
#                         counter += 1
#     return counter

# def index(exams, ecology):
#     with Pool() as pool:
#         results = pool.starmap(calculate_distances, [(row, ecology) for row in exams.itertuples(index=False)])
    
#     total_count = sum(results)
#     return total_count
    
            
def geo_list(geodata):
    if type(geodata) == str:
        try: 
            geodata = eval(geodata)
        except Exeption as e:
            print(geodata)
    geolist = []
    try:
        geolist = list(map(float, str(geodata['coordinates']).replace(',[]', '').replace('[', '').replace(']', '').split(',')))
    except Exception as e:
        print(e, geodata)
    return geolist[0:2]


def dates_set(start, end):
    #start_date = datetime.strptime(start, '%d.%m.%Y')
    #end_date = datetime.strptime(end, '%d.%m.%Y')
    exams_range = pd.date_range(start=start, end=end)
    return set(exams_range)


def sets_intersects(exams_set, start, end):
    """
    Для уменьшения объема датасета ecology, проводим проверку, входят ли даты событий в наборы дат экзаменов
    """
    #date_start = datetime.strptime(start, '%d.%m.%Y')
    #date_end = datetime.strptime(end, '%d.%m.%Y')
    date_range = pd.date_range(start=start, end=end)
    date_set = set(date_range)
    if date_set.intersection(exams_set):
        return True
    else:
        return False

In [3]:
pd.set_option('display.max_colwidth', None) # для отображения полного текста в ячейках
pd.set_option('display.max_columns', None) 
ecology_lite = load_dataset('ecology_lite')
#ecology_lite['start'] = pd.to_datetime(ecology_lite['start'], format='%Y-%m-%d')
#ecology_lite['end'] = pd.to_datetime(ecology_lite['end'], format='%Y-%m-%d')
exams_lite = load_dataset('exams_lite')
#exams_lite['start'] = pd.to_datetime(exams_lite['start'], format='%Y-%m-%d')
#exams_lite['end'] = pd.to_datetime(exams_lite['end'], format='%Y-%m-%d')

In [4]:
#exams_lite['start'] = pd.to_datetime(exams_lite['start'], format='%d.%m.%Y')
#exams_lite['end'] = pd.to_datetime(exams_lite['end'], format='%d.%m.%Y')
exams_lite.head()

,global_id,start,end,stupid,geoData
0,797946035,2017-03-01T00:00:00,2017-09-30T00:00:00,3.800000,"{'type': 'Point', 'coordinates': [37.723652, 55.814547]}"
1,797949150,2017-03-01T00:00:00,2017-09-30T00:00:00,3.357143,"{'coordinates': [[[[37.710083127, 55.825085846], [37.709884644, 55.825088859], [37.70991683, 55.825557425], [37.710107267, 55.825552906], [37.710083127, 55.825085846]]], [[[37.710971466, 55.824666254], [37.710390321, 55.824675741], [37.710401249, 55.824826407], [37.710980407, 55.824816363], [37.710971466, 55.824666254]]], [[[37.710757032, 55.82506651], [37.710557953, 55.825070528], [37.710585371, 55.825566047], [37.710780874, 55.825561359], [37.710757032, 55.82506651]]], [[[37.707049481, 55.825154524], [37.707242202, 55.825149502], [37.707241209, 55.825030085], [37.70704054, 55.82503176], [37.707049481, 55.825154524]]], [[[37.710930347, 55.825726055], [37.711380243, 55.825718823], [37.711373091, 55.825557312], [37.711369514, 55.825539635], [37.710924625, 55.825550081], [37.710924625, 55.825570973], [37.710930347, 55.825726055]]], [[[37.583026114, 55.832647996], [37.582666499, 55.832611174], [37.582629743, 55.832706019], [37.582630736, 55.832758463], [37.58344732, 55.83284717], [37.583486063, 55.832741167], [37.583481096, 55.83270044], [37.583129428, 55.832659155], [37.583156251, 55.832582163], [37.584034426, 55.832677566], [37.584082109, 55.83255092], [37.584084096, 55.832518561], [37.583018166, 55.832398051], [37.582968496, 55.832531393], [37.582976443, 55.832562078], [37.583054923, 55.832573236], [37.583026114, 55.832647996]]], [[[37.709097862, 55.825586152], [37.70911789, 55.825928458], [37.70928669, 55.825930065], [37.709272385, 55.825774179], [37.70981884, 55.825766144], [37.709795952, 55.825566867], [37.709097862, 55.825586152]]]], 'type': 'MultiPolygon'}"
2,797949275,2017-03-01T00:00:00,2017-09-30T00:00:00,1.100000,"{'coordinates': [[[[37.807984129, 55.726758653], [37.808307186, 55.726841896], [37.808526531, 55.726567328], [37.808196321, 55.726480727], [37.807984129, 55.726735157], [37.807984129, 55.726758653]]], [[[37.808976173, 55.726681066], [37.808727265, 55.726995241], [37.808704376, 55.727030686], [37.808858872, 55.727067742], [37.808921814, 55.726979129], [37.808944702, 55.72695174], [37.80899334, 55.726892127], [37.809216499, 55.726948517], [37.80932808, 55.726974296], [37.809522629, 55.72702263], [37.809602738, 55.726898572], [37.809597015, 55.72684057], [37.808976173, 55.726681066]]], [[[37.809706703, 55.726866063], [37.809706703, 55.726887545], [37.809673324, 55.72692044], [37.809673324, 55.726957362], [37.809852138, 55.727005025], [37.810194269, 55.726581425], [37.810197845, 55.726525035], [37.810020223, 55.7264767], [37.809706703, 55.726866063]]], [[[37.808979526, 55.726210184], [37.808632627, 55.726643858], [37.808636203, 55.726692864], [37.808812633, 55.726735828], [37.809155956, 55.726312897], [37.809153572, 55.726253149], [37.808979526, 55.726210184]]]], 'type': 'MultiPolygon'}"
3,797949425,2017-03-01T00:00:00,2017-09-30T00:00:00,6.600000,"{'type': 'Point', 'coordinates': [37.786873, 55.762276]}"
4,797949570,2017-03-01T00:00:00,2017-09-30T00:00:00,5.000000,"{'type': 'Point', 'coordinates': [37.775102, 55.806463]}"


In [5]:
%%time
exams_lite['datesSet'] = exams_lite.apply(lambda x: dates_set(x['start'], x['end']), axis=1)

CPU times: user 3.53 s, sys: 150 ms, total: 3.68 s
Wall time: 3.6 s


In [6]:
exams_lite.head()

,global_id,start,end,stupid,geoData,datesSet
0,797946035,2017-03-01T00:00:00,2017-09-30T00:00:00,3.800000,"{'type': 'Point', 'coordinates': [37.723652, 55.814547]}","{2017-04-02 00:00:00, 2017-03-06 00:00:00, 2017-05-01 00:00:00, 2017-06-23 00:00:00, 2017-07-09 00:00:00, 2017-09-11 00:00:00, 2017-06-03 00:00:00, 2017-07-28 00:00:00, 2017-07-22 00:00:00, 2017-05-02 00:00:00, 2017-03-21 00:00:00, 2017-08-14 00:00:00, 2017-08-02 00:00:00, 2017-09-22 00:00:00, 2017-05-07 00:00:00, 2017-08-28 00:00:00, 2017-03-19 00:00:00, 2017-06-27 00:00:00, 2017-04-05 00:00:00, 2017-08-22 00:00:00, 2017-07-13 00:00:00, 2017-09-26 00:00:00, 2017-06-09 00:00:00, 2017-06-28 00:00:00, 2017-08-09 00:00:00, 2017-07-02 00:00:00, 2017-07-25 00:00:00, 2017-03-31 00:00:00, 2017-07-16 00:00:00, 2017-09-16 00:00:00, 2017-07-20 00:00:00, 2017-04-28 00:00:00, 2017-07-23 00:00:00, 2017-07-24 00:00:00, 2017-09-29 00:00:00, 2017-07-03 00:00:00, 2017-06-29 00:00:00, 2017-04-04 00:00:00, 2017-03-18 00:00:00, 2017-04-15 00:00:00, 2017-03-07 00:00:00, 2017-05-15 00:00:00, 2017-08-27 00:00:00, 2017-03-24 00:00:00, 2017-03-22 00:00:00, 2017-03-30 00:00:00, 2017-05-20 00:00:00, 2017-06-24 00:00:00, 2017-06-18 00:00:00, 2017-03-15 00:00:00, 2017-04-23 00:00:00, 2017-06-14 00:00:00, 2017-05-08 00:00:00, 2017-03-04 00:00:00, 2017-08-18 00:00:00, 2017-09-13 00:00:00, 2017-04-14 00:00:00, 2017-08-21 00:00:00, 2017-07-04 00:00:00, 2017-04-16 00:00:00, 2017-08-08 00:00:00, 2017-06-11 00:00:00, 2017-08-23 00:00:00, 2017-08-10 00:00:00, 2017-08-19 00:00:00, 2017-09-08 00:00:00, 2017-05-05 00:00:00, 2017-05-12 00:00:00, 2017-04-17 00:00:00, 2017-08-17 00:00:00, 2017-04-03 00:00:00, 2017-04-01 00:00:00, 2017-03-20 00:00:00, 2017-07-30 00:00:00, 2017-09-25 00:00:00, 2017-09-30 00:00:00, 2017-05-04 00:00:00, 2017-04-13 00:00:00, 2017-03-25 00:00:00, 2017-06-22 00:00:00, 2017-07-26 00:00:00, 2017-08-03 00:00:00, 2017-09-27 00:00:00, 2017-07-19 00:00:00, 2017-03-05 00:00:00, 2017-06-15 00:00:00, 2017-06-01 00:00:00, 2017-07-06 00:00:00, 2017-04-21 00:00:00, 2017-08-20 00:00:00, 2017-06-25 00:00:00, 2017-03-11 00:00:00, 2017-05-29 00:00:00, 2017-06-05 00:00:00, 2017-07-31 00:00:00, 2017-05-09 00:00:00, 2017-05-22 00:00:00, 2017-09-21 00:00:00, 2017-04-29 00:00:00, 2017-06-20 00:00:00, ...}"
1,797949150,2017-03-01T00:00:00,2017-09-30T00:00:00,3.357143,"{'coordinates': [[[[37.710083127, 55.825085846], [37.709884644, 55.825088859], [37.70991683, 55.825557425], [37.710107267, 55.825552906], [37.710083127, 55.825085846]]], [[[37.710971466, 55.824666254], [37.710390321, 55.824675741], [37.710401249, 55.824826407], [37.710980407, 55.824816363], [37.710971466, 55.824666254]]], [[[37.710757032, 55.82506651], [37.710557953, 55.825070528], [37.710585371, 55.825566047], [37.710780874, 55.825561359], [37.710757032, 55.82506651]]], [[[37.707049481, 55.825154524], [37.707242202, 55.825149502], [37.707241209, 55.825030085], [37.70704054, 55.82503176], [37.707049481, 55.825154524]]], [[[37.710930347, 55.825726055], [37.711380243, 55.825718823], [37.711373091, 55.825557312], [37.711369514, 55.825539635], [37.710924625, 55.825550081], [37.710924625, 55.825570973], [37.710930347, 55.825726055]]], [[[37.583026114, 55.832647996], [37.582666499, 55.832611174], [37.582629743, 55.832706019], [37.582630736, 55.832758463], [37.58344732, 55.83284717], [37.583486063, 55.832741167], [37.583481096, 55.83270044], [37.583129428, 55.832659155], [37.583156251, 55.832582163], [37.584034426, 55.832677566], [37.584082109, 55.83255092], [37.584084096, 55.832518561], [37.583018166, 55.832398051], [37.582968496, 55.832531393], [37.582976443, 55.832562078], [37.583054923, 55.832573236], [37.583026114, 55.832647996]]], [[[37.709097862, 55.825586152], [37.70911789, 55.825928458], [37.70928669, 55.825930065], [37.709272385, 55.825774179], [37.70981884, 55.825766144], [37.709795952, 55.825566867], [37.709097862, 55.825586152]]]], 'type': 'MultiPolygon'}","{2017-04-02 00:00:00, 2017-03-06 00:00:00, 2017-05-01 00:00:00, 2017-06-23 

In [7]:
exams_lite['geoList'] = exams_lite.apply(lambda x: geo_list(x['geoData']), axis=1)

In [8]:
%%time
ecology_lite['datesSet'] = ecology_lite.apply(lambda x: dates_set(x['start'], x['end']), axis=1)

CPU times: user 19.1 s, sys: 732 ms, total: 19.8 s
Wall time: 19.8 s


In [9]:
ecology_lite['geoList'] = ecology_lite.apply(lambda x: geo_list(x['stroi']), axis=1)

In [10]:
ecology_lite.head()

,start,end,stroi,datesSet,geoList
0,2017-01-01T00:00:00,2022-12-31T00:00:00,"{'type': 'Point', 'coordinates': '37.678048127602,55.642035528697'}","{2017-07-09 00:00:00, 2017-11-20 00:00:00, 2021-06-07 00:00:00, 2019-02-17 00:00:00, 2019-03-25 00:00:00, 2018-03-14 00:00:00, 2020-01-09 00:00:00, 2017-06-27 00:00:00, 2020-10-04 00:00:00, 2021-10-19 00:00:00, 2017-03-31 00:00:00, 2019-08-14 00:00:00, 2019-04-07 00:00:00, 2021-05-03 00:00:00, 2021-05-30 00:00:00, 2018-10-11 00:00:00, 2020-06-22 00:00:00, 2019-09-21 00:00:00, 2019-10-20 00:00:00, 2022-06-30 00:00:00, 2022-10-23 00:00:00, 2019-07-04 00:00:00, 2021-04-06 00:00:00, 2018-03-11 00:00:00, 2017-04-16 00:00:00, 2021-09-25 00:00:00, 2022-12-30 00:00:00, 2022-04-13 00:00:00, 2017-08-19 00:00:00, 2022-05-28 00:00:00, 2019-06-15 00:00:00, 2022-09-10 00:00:00, 2019-09-12 00:00:00, 2019-07-21 00:00:00, 2018-02-13 00:00:00, 2022-01-19 00:00:00, 2020-01-30 00:00:00, 2019-04-17 00:00:00, 2021-03-24 00:00:00, 2021-01-14 00:00:00, 2018-04-10 00:00:00, 2018-09-17 00:00:00, 2018-03-23 00:00:00, 2021-01-17 00:00:00, 2018-10-18 00:00:00, 2020-10-23 00:00:00, 2017-01-16 00:00:00, 2019-07-24 00:00:00, 2021-11-19 00:00:00, 2019-02-13 00:00:00, 2022-10-10 00:00:00, 2020-09-27 00:00:00, 2017-11-26 00:00:00, 2020-07-25 00:00:00, 2018-03-10 00:00:00, 2017-02-11 00:00:00, 2021-12-22 00:00:00, 2020-09-09 00:00:00, 2018-03-22 00:00:00, 2022-10-21 00:00:00, 2017-11-24 00:00:00, 2020-09-11 00:00:00, 2022-07-06 00:00:00, 2022-11-27 00:00:00, 2021-06-17 00:00:00, 2019-12-03 00:00:00, 2021-01-27 00:00:00, 2020-09-12 00:00:00, 2022-02-12 00:00:00, 2021-07-14 00:00:00, 2017-02-15 00:00:00, 2019-06-17 00:00:00, 2022-07-31 00:00:00, 2017-10-26 00:00:00, 2017-08-25 00:00:00, 2022-12-13 00:00:00, 2018-11-14 00:00:00, 2021-05-08 00:00:00, 2019-10-12 00:00:00, 2020-05-26 00:00:00, 2022-02-19 00:00:00, 2022-08-02 00:00:00, 2021-12-06 00:00:00, 2018-03-19 00:00:00, 2019-09-14 00:00:00, 2017-10-21 00:00:00, 2019-10-27 00:00:00, 2022-06-07 00:00:00, 2017-08-29 00:00:00, 2022-04-08 00:00:00, 2022-06-05 00:00:00, 2018-11-04 00:00:00, 2019-01-11 00:00:00, 2019-06-08 00:00:00, 2021-01-19 00:00:00, 2019-06-24 00:00:00, 2020-06-08 00:00:00, 2022-05-24 00:00:00, 2022-08-17 00:00:00, 2019-03-29 00:00:00, ...}","[37.678048127602, 55.642035528697]"
1,2017-01-01T00:00:00,2023-12-31T00:00:00,"{'type': 'Point', 'coordinates': '37.610077770368,55.622021730289'}","{2023-04-02 00:00:00, 2017-07-09 00:00:00, 2017-11-20 00:00:00, 2021-06-07 00:00:00, 2023-09-26 00:00:00, 2023-11-28 00:00:00, 2019-02-17 00:00:00, 2023-01-10 00:00:00, 2023-11-21 00:00:00, 2019-03-25 00:00:00, 2018-03-14 00:00:00, 2020-01-09 00:00:00, 2017-06-27 00:00:00, 2020-10-04 00:00:00, 2023-07-02 00:00:00, 2021-10-19 00:00:00, 2017-03-31 00:00:00, 2019-08-14 00:00:00, 2019-04-07 00:00:00, 2021-05-03 00:00:00, 2021-05-30 00:00:00, 2023-03-02 00:00:00, 2018-10-11 00:00:00, 2020-06-22 00:00:00, 2019-09-21 00:00:00, 2019-10-20 00:00:00, 2023-07-19 00:00:00, 2022-06-30 00:00:00, 2022-10-23 00:00:00, 2023-05-25 00:00:00, 2019-07-04 00:00:00, 2021-04-06 00:00:00, 2023-12-26 00:00:00, 2018-03-11 00:00:00, 2017-04-16 00:00:00, 2021-09-25 00:00:00, 2022-12-30 00:00:00, 2022-04-13 00:00:00, 2017-08-19 00:00:00, 2023-06-15 00:00:00, 2022-05-28 00:00:00, 2019-06-15 00:00:00, 2023-04-16 00:00:00, 2022-09-10 00:00:00, 2019-09-12 00:00:00, 2023-05-16 00:00:00, 2019-07-21 00:00:00, 2023-09-28 00:00:00, 2023-06-23 00:00:00, 2018-02-13 00:00:00, 2022-01-19 00:00:00, 2023-03-25 00:00:00, 2020-01-30 00:00:00, 2019-04-17 00:00:00, 2021-03-24 00:00:00, 2021-01-14 00:00:00, 2018-04-10 00:00:00, 2018-09-17 00:00:00, 2018-03-23 00:00:00, 2021-01-17 00:00:00, 2018-10-18 00:00:00, 2020-10-23 00:00:00, 2017-01-16 00:00:00, 2019-07-24 00:00:00, 2021-11-19 00:00:00, 2019-02-13 00:00:00, 2022-10-10 00:00:00, 2020-09-27 00:00:00, 2017-11-26 00:00:00, 2020-07-25 00:00:00, 2018-03-10 00:00:00, 2017-02-11 00:00:00, 2021-12-22 00:00:00, 2020-09-09 00:00:00, 2018-03-22 00:00:00, 2022-10-21 00:00:00,

In [15]:
%%time
exams_lite['near_stroi'] = exams_lite.apply(lambda x: km_min(x, ecology_lite), axis=1)

CPU times: user 8min 50s, sys: 26.1 ms, total: 8min 50s
Wall time: 8min 52s


In [17]:
exams_stroi = exams_lite[['global_id', 'start', 'end', 'near_stroi', 'stupid']]
# exams_lite.apply(lambda x: index(x, ecology_lite), axis=1)
#result = index(exams_lite, ecology_lite)
#%%time
#km_min(exams_lite.iloc[6], ecology_lite)
save_dataset('exams_stroi')

Сохранено 5542 записей в exams_stroi.gz
